In [1]:
import numpy as np
from ase import Atoms
from ase.visualize import view

def generate_supercell_positions(unit_cell_positions, n):
    """
    Generate positions for the supercell based on unit cell positions and repetitions.
    """
    supercell_positions = []
    symbols = []
    for x in range(n):
        for y in range(n):
            for z in range(n):
                translation_vector = np.array([x, y, z])
                for element, positions in unit_cell_positions.items():
                    for pos in positions:
                        new_pos = (pos + translation_vector) / n
                        supercell_positions.append(new_pos)
                        symbols.append(element)
    return supercell_positions, symbols

def read_atomic_positions(filename):
    """
    Reads atomic positions from a preprocessed scf.in file and returns them in a dictionary format.

    Args:
        filename (str): Path to the scf.in file

    Returns:
        dict: Dictionary with atomic species as keys and lists of numpy arrays of positions as values
    """
    unit_cell_positions = {}
    
    with open(filename, 'r') as f:
        in_positions_section = False
        for line in f:
            if line.startswith('ATOMIC_POSITIONS'):
                in_positions_section = True
            elif in_positions_section and line.strip() == '':
                in_positions_section = False
            elif in_positions_section:
                data = line.split()
                if len(data) >= 4 and data[0].isalpha():
                    symbol = data[0]
                    try:
                        x, y, z = map(float, data[1:4])
                        if symbol not in unit_cell_positions:
                            unit_cell_positions[symbol] = []
                        unit_cell_positions[symbol].append(np.array([x, y, z]))
                    except ValueError:
                        print(f"Warning: Skipping line with invalid format: {line}")
    
    return unit_cell_positions

def find_reference_as(symbols, positions):
    """
    Find the first As atom to use as a reference point.
    """
    for symbol, pos in zip(symbols, positions):
        if symbol == "As":
            return pos
    return None

def find_as_110_positions(reference_as_pos, symbols, positions):
    """
    Identify As atoms along the [110] direction based on the reference As atom.
    """
    as_110_positions = []
    as_110_indices = []
    print(f'As along [110] direction with reference {reference_as_pos}:')
    if reference_as_pos is not None:
        for i, (symbol, pos) in enumerate(zip(symbols, positions)):
            if symbol == "As":
                diff = pos - reference_as_pos
                diff = np.mod(diff + 1, 1)  # Normalize to [0,1) range
                diff_rounded = np.round(diff, 6)  # Handle floating point precision issues
                if (diff_rounded == [0.25, 0.25, 0]).all() or \
                   (diff_rounded == [0.50, 0.50, 0]).all() or \
                   (diff_rounded == [0.75, 0.75, 0]).all():
                    print(f'{pos}')
                    as_110_positions.append(pos)
                    as_110_indices.append(i)

    if not as_110_indices:
        print('No As along [110] direction')
            
                    
    return as_110_positions, as_110_indices

def replace_first_as_with_bi(symbols, as_110_indices):
    """
    Replace the first As atom along the [110] direction with Bi.
    """
    if as_110_indices:
        first_as_110_index = as_110_indices[0]
        symbols[first_as_110_index] = "Bi"
    return symbols


def get_final_positions(symbols, positions):
    """
    Get the final atomic positions in a format required for Quantum ESPRESSO.
    
    Args:
        symbols (list): List of atomic symbols.
        positions (list): List of atomic positions (fractional coordinates).
    
    Returns:
        list: List of tuples containing atomic symbols and positions.
    """
    atomic_positions = []
    for symbol, pos in zip(symbols, positions):
        atomic_positions.append((symbol, pos))
    #Print the new coordinates:
    print("Final atomic positions (fractional coordinates):")
    print("ATOMIC_POSITIONS crystal")
    for symbol, pos in zip(symbols, positions):
        print(f"{symbol} {pos[0]:.6f} {pos[1]:.6f} {pos[2]:.6f}")
    return atomic_positions

def write_new_scf(atomic_positions, original_filename, n):
    """
    Write a new scf input file with updated atomic positions and cell parameters.

    Args:
        atomic_positions (list): List of tuples containing atomic symbols and positions.
        original_filename (str): Path to the original scf.in file.
        n (int): Number of repetitions in each direction for the supercell.
    """
    new_filename = f'scf_{n}{n}{n}.in'

    with open(original_filename, 'r') as original_file:
        lines = original_file.readlines()

    in_positions_section = False
    new_lines = []

    # Variables to store cell parameters
    celldm = None
    nat = None

    for line in lines:
        if line.startswith('ATOMIC_POSITIONS'):
            in_positions_section = True
            new_lines.append(line)
            for symbol, pos in atomic_positions:
                new_lines.append(f"{symbol} {pos[0]:.6f} {pos[1]:.6f} {pos[2]:.6f}\n")
        elif line.startswith('celldm(1)') or line.startswith('a'):
            # Parse celldm(1) or 'a' value and multiply by n
            celldm_value = float(line.split('=')[1].split()[0])
            celldm = celldm_value * n
            new_lines.append(f'celldm(1) = {celldm}\n')
        elif line.startswith('nat'):
            # Parse 'nat' value and multiply by n
            nat_value = int(line.split('=')[1])
            nat = nat_value * n**3
            new_lines.append(f'nat = {nat}\n')
        elif in_positions_section and line.strip() == '':
            in_positions_section = False
        elif not in_positions_section:
            new_lines.append(line)

    with open(new_filename, 'w') as new_file:
        new_file.writelines(new_lines)

def main():
    unit_cell_positions = read_atomic_positions(filename)
    # Step 1: Generate supercell positions
    supercell_positions, symbols = generate_supercell_positions(unit_cell_positions, n)
    
    # Step 2: Find the first As atom as the reference
    reference_as_pos = find_reference_as(symbols, supercell_positions)
    
    # Step 3: Identify As atoms along the [110] direction
    as_110_positions, as_110_indices = find_as_110_positions(reference_as_pos, symbols, supercell_positions)
    
    # Step 4: Replace the first As along [110] with Bi
    symbols = replace_first_as_with_bi(symbols, as_110_indices)
    
    # Step 5: Get the final atomic positions
    atomic_positions = get_final_positions(symbols, supercell_positions)
    
    # Step 6: Write the new scf input file with updated coordinates
    write_new_scf(atomic_positions, filename, n)
    
    # Step 7: Visualize the supercell
    supercell = Atoms(symbols=symbols, scaled_positions=supercell_positions, cell=[n*a, n*a, n*a], pbc=True)
    #view(supercell)

    
# Define the lattice constant and number of repetitions
a = 5.61
n = 4
filename = 'GaAs/LDA/super_cell_444/GaAs-scf.in'
if __name__ == "__main__":
    main()



As along [110] direction with reference [0.0625 0.0625 0.0625]:
[0.3125 0.3125 0.0625]
[0.5625 0.5625 0.0625]
[0.8125 0.8125 0.0625]
Final atomic positions (fractional coordinates):
ATOMIC_POSITIONS crystal
Ga 0.000000 0.000000 0.000000
As 0.062500 0.062500 0.062500
Ga 0.000000 0.000000 0.250000
As 0.062500 0.062500 0.312500
Ga 0.000000 0.000000 0.500000
As 0.062500 0.062500 0.562500
Ga 0.000000 0.000000 0.750000
As 0.062500 0.062500 0.812500
Ga 0.000000 0.250000 0.000000
As 0.062500 0.312500 0.062500
Ga 0.000000 0.250000 0.250000
As 0.062500 0.312500 0.312500
Ga 0.000000 0.250000 0.500000
As 0.062500 0.312500 0.562500
Ga 0.000000 0.250000 0.750000
As 0.062500 0.312500 0.812500
Ga 0.000000 0.500000 0.000000
As 0.062500 0.562500 0.062500
Ga 0.000000 0.500000 0.250000
As 0.062500 0.562500 0.312500
Ga 0.000000 0.500000 0.500000
As 0.062500 0.562500 0.562500
Ga 0.000000 0.500000 0.750000
As 0.062500 0.562500 0.812500
Ga 0.000000 0.750000 0.000000
As 0.062500 0.812500 0.062500
Ga 0.000000 0

In [2]:
import numpy as np
from ase import Atoms
from ase.visualize import view

def generate_supercell_positions(unit_cell_positions, n):
    """
    Generate positions for the supercell based on unit cell positions and repetitions.
    """
    supercell_positions = []
    symbols = []
    for x in range(n):
        for y in range(n):
            for z in range(n):
                translation_vector = np.array([x, y, z])
                for element, positions in unit_cell_positions.items():
                    for pos in positions:
                        new_pos = (pos + translation_vector) / n
                        supercell_positions.append(new_pos)
                        symbols.append(element)
    return supercell_positions, symbols

def read_atomic_positions(filename):
    """
    Reads atomic positions from a preprocessed scf.in file and returns them in a dictionary format.

    Args:
        filename (str): Path to the scf.in file

    Returns:
        dict: Dictionary with atomic species as keys and lists of numpy arrays of positions as values
    """
    unit_cell_positions = {}
    
    with open(filename, 'r') as f:
        in_positions_section = False
        for line in f:
            if line.startswith('ATOMIC_POSITIONS'):
                in_positions_section = True
            elif in_positions_section and line.strip() == '':
                in_positions_section = False
            elif in_positions_section:
                data = line.split()
                if len(data) >= 4 and data[0].isalpha():
                    symbol = data[0]
                    try:
                        x, y, z = map(float, data[1:4])
                        if symbol not in unit_cell_positions:
                            unit_cell_positions[symbol] = []
                        unit_cell_positions[symbol].append(np.array([x, y, z]))
                    except ValueError:
                        print(f"Warning: Skipping line with invalid format: {line}")
    
    return unit_cell_positions

def find_reference_as(symbols, positions):
    """
    Find the first As atom to use as a reference point.
    """
    for symbol, pos in zip(symbols, positions):
        if symbol == "As":
            return pos
    return None

def find_as_110_positions(reference_as_pos, symbols, positions):
    """
    Identify As atoms along the [110] direction based on the reference As atom.
    """
    as_110_positions = []
    as_110_indices = []
    print(f'As along [110] direction with reference {reference_as_pos}:')
    if reference_as_pos is not None:
        for i, (symbol, pos) in enumerate(zip(symbols, positions)):
            if symbol == "As":
                diff = pos - reference_as_pos
                diff = np.mod(diff + 1, 1)  # Normalize to [0,1) range
                diff_rounded = np.round(diff, 6)  # Handle floating point precision issues
                if (diff_rounded == [0.25, 0.25, 0]).all() or \
                   (diff_rounded == [0.50, 0.50, 0]).all() or \
                   (diff_rounded == [0.75, 0.75, 0]).all():
                    print(f'{pos}')
                    as_110_positions.append(pos)
                    as_110_indices.append(i)

    if not as_110_indices:
        print('No As along [110] direction')
            
                    
    return as_110_positions, as_110_indices

def replace_first_as_with_bi(symbols, as_110_indices):
    """
    Replace the first As atom along the [110] direction with Bi.
    """
    if as_110_indices:
        first_as_110_index = as_110_indices[0]
        second_as_110_index = as_110_indices[1]
        third_as_110_index = as_110_indices[2]
        symbols[first_as_110_index] = "Bi"
        symbols[second_as_110_index] = "Bi"
        symbols[third_as_110_index] = "Bi"
    return symbols


def get_final_positions(symbols, positions):
    """
    Get the final atomic positions in a format required for Quantum ESPRESSO.
    
    Args:
        symbols (list): List of atomic symbols.
        positions (list): List of atomic positions (fractional coordinates).
    
    Returns:
        list: List of tuples containing atomic symbols and positions.
    """
    atomic_positions = []
    for symbol, pos in zip(symbols, positions):
        atomic_positions.append((symbol, pos))
    #Print the new coordinates:
    print("Final atomic positions (fractional coordinates):")
    print("ATOMIC_POSITIONS crystal")
    for symbol, pos in zip(symbols, positions):
        print(f"{symbol} {pos[0]:.6f} {pos[1]:.6f} {pos[2]:.6f}")
    return atomic_positions

def write_new_scf(atomic_positions, original_filename, n):
    """
    Write a new scf input file with updated atomic positions and cell parameters.

    Args:
        atomic_positions (list): List of tuples containing atomic symbols and positions.
        original_filename (str): Path to the original scf.in file.
        n (int): Number of repetitions in each direction for the supercell.
    """
    new_filename = f'scf_{n}{n}{n}.in'

    with open(original_filename, 'r') as original_file:
        lines = original_file.readlines()

    in_positions_section = False
    new_lines = []

    # Variables to store cell parameters
    celldm = None
    nat = None

    for line in lines:
        if line.startswith('ATOMIC_POSITIONS'):
            in_positions_section = True
            new_lines.append(line)
            for symbol, pos in atomic_positions:
                new_lines.append(f"{symbol} {pos[0]:.6f} {pos[1]:.6f} {pos[2]:.6f}\n")
        elif line.startswith('celldm(1)') or line.startswith('a'):
            # Parse celldm(1) or 'a' value and multiply by n
            celldm_value = float(line.split('=')[1].split()[0])
            celldm = celldm_value * n
            new_lines.append(f'celldm(1) = {celldm}\n')
        elif line.startswith('nat'):
            # Parse 'nat' value and multiply by n
            nat_value = int(line.split('=')[1])
            nat = nat_value * n**3
            new_lines.append(f'nat = {nat}\n')
        elif in_positions_section and line.strip() == '':
            in_positions_section = False
        elif not in_positions_section:
            new_lines.append(line)

    with open(new_filename, 'w') as new_file:
        new_file.writelines(new_lines)

def main():
    unit_cell_positions = read_atomic_positions(filename)
    # Step 1: Generate supercell positions
    supercell_positions, symbols = generate_supercell_positions(unit_cell_positions, n)
    
    # Step 2: Find the first As atom as the reference
    reference_as_pos = find_reference_as(symbols, supercell_positions)
    
    # Step 3: Identify As atoms along the [110] direction
    as_110_positions, as_110_indices = find_as_110_positions(reference_as_pos, symbols, supercell_positions)
    
    # Step 4: Replace the first As along [110] with Bi
    symbols = replace_first_as_with_bi(symbols, as_110_indices)
    
    # Step 5: Get the final atomic positions
    atomic_positions = get_final_positions(symbols, supercell_positions)
    
    # Step 6: Write the new scf input file with updated coordinates
    write_new_scf(atomic_positions, filename, n)
    
    # Step 7: Visualize the supercell
    supercell = Atoms(symbols=symbols, scaled_positions=supercell_positions, cell=[n*a, n*a, n*a], pbc=True)
    #view(supercell)

    
# Define the lattice constant and number of repetitions
a = 5.61
n = 4
filename = '../scf.in'
if __name__ == "__main__":
    main()



As along [110] direction with reference [0.0625 0.0625 0.0625]:
[0.3125 0.3125 0.0625]
[0.5625 0.5625 0.0625]
[0.8125 0.8125 0.0625]
Final atomic positions (fractional coordinates):
ATOMIC_POSITIONS crystal
Ga 0.000000 0.000000 0.000000
As 0.062500 0.062500 0.062500
Ga 0.000000 0.000000 0.250000
As 0.062500 0.062500 0.312500
Ga 0.000000 0.000000 0.500000
As 0.062500 0.062500 0.562500
Ga 0.000000 0.000000 0.750000
As 0.062500 0.062500 0.812500
Ga 0.000000 0.250000 0.000000
As 0.062500 0.312500 0.062500
Ga 0.000000 0.250000 0.250000
As 0.062500 0.312500 0.312500
Ga 0.000000 0.250000 0.500000
As 0.062500 0.312500 0.562500
Ga 0.000000 0.250000 0.750000
As 0.062500 0.312500 0.812500
Ga 0.000000 0.500000 0.000000
As 0.062500 0.562500 0.062500
Ga 0.000000 0.500000 0.250000
As 0.062500 0.562500 0.312500
Ga 0.000000 0.500000 0.500000
As 0.062500 0.562500 0.562500
Ga 0.000000 0.500000 0.750000
As 0.062500 0.562500 0.812500
Ga 0.000000 0.750000 0.000000
As 0.062500 0.812500 0.062500
Ga 0.000000 0

In [3]:
10.601362796*4

42.405451184

In [4]:
3/64*100

4.6875